In [1]:
import numpy as np
import pandas as pd
import yfinance
import ta

# Base Strategy
The base strategy for a simple stochrsi strategy:
### Part 1: Buying Requirements
  1. The close price must be above the 200 sma (Simple Moving Average)
  2. The Stochastic RSI K must below certain percentage (n%)
  3. Placing a n % limit buy order the next period with an order limit of n periods
### Part 2: Selling Requirements
  1. The trailing stop is equal to n% of the limit buy percentage
  2. Otherwise sell at trailing stop

In [36]:
# Starting with the analysis with AAPL
# download AAPL's daily price from 2010-01-01 to 2020-12-31 (10 years)
symbol = 'AAPL'
ticker = yfinance.Ticker(symbol)
prices = ticker.history(start='2010-01-01', end='2020-12-31')

In [37]:
prices

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2010-01-04,6.508332,6.540961,6.476314,6.526019,493729600,0.0,0.0
2010-01-05,6.544012,6.574201,6.502845,6.537303,601904800,0.0,0.0
2010-01-06,6.537304,6.563224,6.426610,6.433319,552160000,0.0,0.0
2010-01-07,6.457105,6.464728,6.374771,6.421426,477131200,0.0,0.0
2010-01-08,6.412888,6.464728,6.375076,6.464118,447610800,0.0,0.0
...,...,...,...,...,...,...,...
2020-12-23,130.808961,131.076190,129.443063,129.621231,88223700,0.0,0.0
2020-12-24,129.977561,132.095684,129.759809,130.620911,54930100,0.0,0.0
2020-12-28,132.620253,135.935997,132.145149,135.292648,124486200,0.0,0.0


In [38]:
# first getting the 200 SMA (Simple Moving Average)
sma200 = ta.trend.sma_indicator(prices.Close)
sma200

Date
2010-01-04           NaN
2010-01-05           NaN
2010-01-06           NaN
2010-01-07           NaN
2010-01-08           NaN
                 ...    
2020-12-23    125.017934
2020-12-24    125.643970
2020-12-28    126.873768
2020-12-29    127.833028
2020-12-30    128.765893
Name: sma_12, Length: 2768, dtype: float64

In [39]:
# Second getting the stockastic 
stoch = ta.momentum.stochrsi_k(prices.Close, window=10)
stoch

Date
2010-01-04         NaN
2010-01-05         NaN
2010-01-06         NaN
2010-01-07         NaN
2010-01-08         NaN
                ...   
2020-12-23    0.823846
2020-12-24    0.893415
2020-12-28    0.893415
2020-12-29    0.808880
2020-12-30    0.597016
Name: stochrsi_k, Length: 2768, dtype: float64

In [40]:
# checking if close is above 200 sma and stoch < n%
signal = (prices.Close > sma200) & (stoch < .2)
signal

Date
2010-01-04    False
2010-01-05    False
2010-01-06    False
2010-01-07    False
2010-01-08    False
              ...  
2020-12-23    False
2020-12-24    False
2020-12-28    False
2020-12-29    False
2020-12-30    False
Length: 2768, dtype: bool

In [41]:
# Create new df with close price and signal
ndf = pd.concat([prices.Close, signal], axis=1)
ndf.columns = ['Close', 'Signal']
ndf

,Close,Signal
Date,,
2010-01-04,6.526019,False
2010-01-05,6.537303,False
2010-01-06,6.433319,False
2010-01-07,6.421426,False
2010-01-08,6.464118,False
...,...,...
2020-12-23,129.621231,False
2020-12-24,130.620911,False
2020-12-28,135.292648,False


In [42]:
# Limit order when there is a signal at 97% of the current close price
buy_limit_price = ndf.loc[ndf.Signal, 'Close'] * .90
buy_limit_price

Date
2010-03-22     6.168174
2010-04-20     6.712674
2010-04-29     7.372719
2010-04-30     7.165509
2010-05-03     7.309870
                ...    
2020-01-24    70.280647
2020-01-28    70.143764
2020-02-04    70.399883
2020-06-15    76.115039
2020-07-21    86.103486
Name: Close, Length: 91, dtype: float64

In [43]:
# Shifting the index of the buy price by 1
idx = pd.Series(np.arange(len(ndf)), index=ndf.index)
idx = idx.shift(-1).dropna().loc[ndf.Signal].astype(int)
idx

Date
2010-03-22      54
2010-04-20      74
2010-04-29      81
2010-04-30      82
2010-05-03      83
              ... 
2020-01-24    2532
2020-01-28    2534
2020-02-04    2539
2020-06-15    2630
2020-07-21    2655
Length: 91, dtype: int32

In [44]:
# loop through signal and check the next 10 days 
# buy when the price is meet
# for stocks with good momemtum 
# sell with trailing stop of n% of the drop percetage (in this case 10%)
trail_stop_pct = 1 - .1 * .6
win_ratio = []
max_buy_period = 20
current_i = 0
for i, bp in zip(idx, buy_limit_price):
  # Check if still holding 
  if current_i >= i:
    continue
  s, e = i, i + max_buy_period
  data = prices.iloc[i: e, 1: 4]
  # Check if the price is lower than buy_limit_price
  has_brought = (data < bp).sum(1)
  if has_brought.sum() == 0:
    # if not brought continue
    continue
  # Since brought moving to sell logic
  buy_date = has_brought.loc[(has_brought > 0)].index[0]
  sect_data = prices.loc[buy_date:, ['Open', 'High', 'Low', 'Close']]
  # Second loop 
  # looping through all prices to find selling date
  stop_price = sect_data.High.iloc[0] * trail_stop_pct
  current_high = sect_data.High.iloc[0]
  for ii, (d, row) in enumerate(sect_data.iloc[1:].iterrows()):
    # check sell
    if row.Open < stop_price:
      # Sell logics
      wp = row.Open / bp - 1
      win_ratio.append(wp)
      current_i = ii + 1 + i
      break
    elif row.Low < stop_price:
      # Sell logics
      wp = stop_price / bp - 1
      win_ratio.append(wp)
      current_i = ii + 1 + i
      break
    # Increasing the stop_price when there is a new high
    if current_high < row.High:
      stop_price = row.High * stop_price

In [45]:
# Checking the strategy gain
strategy_gain = np.prod(np.array(win_ratio) + 1) - 1
print(f'This Strategy has a gaining rate of : {strategy_gain*100:.2f}%') 

This Strategy has a gaining rate of : 47.91%


In [46]:
# Seeing how much can be gain through moment
static_gain = prices.Close[-1] / prices.Close[0] - 1
print(f'The static gain of {symbol} is :  {static_gain*100:.2f}%') 

The static gain of AAPL is :  1928.08%


In [47]:
# Seeing how much can be gain when sell at the highest price
highest_gain = prices.High.max() / prices.Close[0] - 1
print(f'The maximum gain of {symbol} is :  {highest_gain*100:.2f}%') 

The maximum gain of AAPL is :  2004.98%


The result does not seem to be ideal. Let see how can we improve it.